# Recommendation systems
### We use 3 differents recomendation systems types to find similar result and get the best decision to apply

*  Surprise
*  Multilabel Encoder - TFIDF
*  TFIDF and cosine similarity




---

In [73]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import seaborn as sns
from nltk import FreqDist
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.collocations import *
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from wordcloud import WordCloud, STOPWORDS 
from bs4 import BeautifulSoup
from PIL import Image
from surprise import Dataset
from surprise import Reader
import requests
import gzip
import pandas as pd
import gc #garbage collector
sns.set()
reader = Reader()

In [74]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

**Reviews**

In [75]:
contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n = 3, 
# abrira uno de cada tres, y asi.
n = 10
for l in parse('steam_reviews.json.gz'):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1

In [76]:
data_reviews = pd.DataFrame(data_reviews)

In [77]:
data_reviews.head()

,username,hours,products,product_id,page_order,date,text,early_access,page,user_id,compensation,found_funny
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,Ariman1,13.2,1386.0,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,NaN,NaN
2,freakfantom,0.1,1706.0,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,NaN,NaN
3,The_Cpt_FROGGY,7.8,2217.0,631920,0,2017-12-11,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,False,1,76561198003030375,Product received for free,NaN
4,the_maker988,8.2,18.0,35140,7,2018-01-02,"This game was way to linear for me, and compar...",False,1,76561198353396844,NaN,NaN


In [78]:
data_reviews.isna().sum()

username             0
hours             2637
products          1566
product_id           0
page_order           0
date                 0
text                 0
early_access         0
page                 0
user_id         461967
compensation    764719
found_funny     659143
dtype: int64

In [79]:
data_reviews.describe()

,hours,products,page_order,page,found_funny
count,776670.000000,777741.000000,779307.000000,779307.000000,120164.000000
mean,111.843832,236.993936,4.417743,893.581355,7.713125
std,390.145376,486.034479,2.897848,1928.272966,72.465816
min,0.000000,1.000000,0.000000,1.000000,1.000000
25%,4.000000,45.000000,2.000000,53.000000,1.000000
50%,15.300000,110.000000,4.000000,238.000000,1.000000
75%,59.600000,246.000000,7.000000,829.000000,3.000000
max,28164.000000,13540.000000,9.000000,18371.000000,6524.000000


In [80]:
data_reviews.columns = [ 'username','hours','products','id','page_order','date','text','early_access','page','user_id','compensation','found_funny']

In [81]:
data_reviews

,username,hours,products,id,page_order,date,text,early_access,page,user_id,compensation,found_funny
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,Ariman1,13.2,1386.0,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,NaN,NaN
2,freakfantom,0.1,1706.0,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,NaN,NaN
3,The_Cpt_FROGGY,7.8,2217.0,631920,0,2017-12-11,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,False,1,76561198003030375,Product received for free,NaN
4,the_maker988,8.2,18.0,35140,7,2018-01-02,"This game was way to linear for me, and compar...",False,1,76561198353396844,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
779302,Vidaar,783.5,353.0,252490,1,2013-12-11,Summary: Rust is a survival game created by Fa...,True,10217,NaN,NaN,NaN
779303,Nikolai Belinski,55.1,204.0,252490,1,2013-12-11,Coolest Game Eva,True,10218,NaN,NaN,NaN
779304,RancorZealot,203.5,173.0,252490,1,2013-12-11,Should you buy RUST? Uh... YES. The BEST Survi...,True,10219,76561197963920623,NaN,NaN
779305,Jerry,139.8,79.0,252490,1,2013-12-11,Great game!,True,10220,76561198042664856,NaN,NaN


---

**Games**

In [82]:
data_games = []
for l in parse('steam_games.json.gz'):
    data_games.append(l)
data_games = pd.DataFrame(data_games)

In [83]:
data_games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [84]:
data_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN
32134,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",NaN,http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN,Positive,NaN


In [85]:
data_games.isna().sum()

publisher          8052
genres             3283
app_name              2
title              2050
url                   0
release_date       2067
tags                163
discount_price    31910
reviews_url           2
specs               670
price              1377
early_access          0
id                    2
developer          3299
sentiment          7182
metascore         29458
dtype: int64

In [86]:
data_games.describe()

,discount_price
count,225.000000
mean,11.930533
std,17.492643
min,0.490000
25%,1.390000
50%,4.190000
75%,22.660000
max,139.990000


---

In [87]:
df = pd.merge(data_reviews, data_games, on=["id"])
df

,username,hours,products,id,page_order,date,text,early_access_x,page,user_id,...,release_date,tags,discount_price,reviews_url,specs,price,early_access_y,developer,sentiment,metascore
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,...,2017-11-10,"[Action, Adventure, Indie, Simulation]",NaN,http://steamcommunity.com/app/725280/reviews/?...,"[Single-player, Steam Trading Cards]",2.99,False,Vyacheslav Shilikhin,Positive,NaN
1,freakfantom,0.1,1706.0,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,...,2017-11-10,"[Action, Adventure, Indie, Simulation]",NaN,http://steamcommunity.com/app/725280/reviews/?...,"[Single-player, Steam Trading Cards]",2.99,False,Vyacheslav Shilikhin,Positive,NaN
2,Ariman1,13.2,1386.0,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN
3,dll3z,0.4,314.0,328100,5,2017-03-07,do not buy this game,False,2,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN
4,Nyazicat,2.2,1224.0,328100,1,2017-01-09,For some reason I entirely derped out and boug...,False,3,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779946,rt974,9.4,36.0,518790,3,2017-02-16,"Hi,\nI discovered this game yesterday on my pe...",False,381,76561198032901419,...,2017-02-16,"[Hunting, Open World, Multiplayer, Simulation,...",NaN,http://steamcommunity.com/app/518790/reviews/?...,"[Single-player, Multi-player, Steam Achievemen...",29.99,False,Expansive Worlds,Mostly Positive,NaN
779947,Fallenglory,35.8,606.0,518790,3,2017-02-16,The Hunter was and is still the most realistic...,False,382,76561197992526609,...,2017-02-16,"[Hunting, Open World, Multiplayer, Simulation,...",NaN,http://steamcommunity.com/app/518790/reviews/?...,"[Single-player, Multi-player, Steam Achievemen...",29.99,False,Expansive Worlds,Mostly Positive,NaN
779948,GooseMan,32.4,166.0,518790,7,2017-02-16,Very good!,False,383,76561198037224895,...,2017-02-16,"[Hunting, Open World, Multiplayer, Simulation,...",NaN,http://steamcommunity.com/app/518790/reviews/?...,"[Single-player, Multi-player, Steam Achievemen...",29.99,False,Expansive Worlds,Mostly Positive,NaN
779949,lazyboy82,96.9,22.0,518790,3,2017-02-16,Update after 60+ hours:\nI decided to quit the...,False,384,NaN,...,2017-02-16,"[Hunting, Open World, Multiplayer, Simulation,...",NaN,http://steamcommunity.com/app/518790/reviews/?...,"[Single-player, Multi-player, Steam Achievemen...",29.99,False,Expansive Worlds,Mostly Positive,NaN


In [88]:
df.head()

,username,hours,products,id,page_order,date,text,early_access_x,page,user_id,...,release_date,tags,discount_price,reviews_url,specs,price,early_access_y,developer,sentiment,metascore
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,...,2017-11-10,"[Action, Adventure, Indie, Simulation]",NaN,http://steamcommunity.com/app/725280/reviews/?...,"[Single-player, Steam Trading Cards]",2.99,False,Vyacheslav Shilikhin,Positive,NaN
1,freakfantom,0.1,1706.0,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,...,2017-11-10,"[Action, Adventure, Indie, Simulation]",NaN,http://steamcommunity.com/app/725280/reviews/?...,"[Single-player, Steam Trading Cards]",2.99,False,Vyacheslav Shilikhin,Positive,NaN
2,Ariman1,13.2,1386.0,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN
3,dll3z,0.4,314.0,328100,5,2017-03-07,do not buy this game,False,2,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN
4,Nyazicat,2.2,1224.0,328100,1,2017-01-09,For some reason I entirely derped out and boug...,False,3,NaN,...,2014-11-07,"[Adventure, RPG, Indie, Puzzle, Match 3]",NaN,http://steamcommunity.com/app/328100/reviews/?...,[Single-player],14.99,False,Ino-Co Plus,Mixed,NaN


___

# Convertimos los usernames de str a int

In [89]:
df_0 = pd.DataFrame(df.username.str.replace(' ', ''))

In [90]:
df_0.head()

,username
0,ChaosSyren
1,freakfantom
2,Ariman1
3,dll3z
4,Nyazicat


In [91]:
df_1 = pd.DataFrame(df_0.username.astype("category").cat.codes)

In [92]:
df_1.columns = ['users']

In [93]:
df_1.head()

,users
0,62225
1,431962
2,26750
3,422892
4,245491


In [94]:
df_1.tail()

,users
779946,485826
779947,115225
779948,136213
779949,456420
779950,255487


---

# Acotamos el Dataset a las categorías que vamos a utilizar

In [95]:
df_2 = df[['hours','id','app_name','tags']]

In [96]:
df_2.head()

,hours,id,app_name,tags
0,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,13.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,0.4,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,2.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


---

* Usamos el code para ver si referencia bien el ID con las peliculas

In [97]:
game_id = 432
print(df_2.loc[game_id].app_name)
print(df_2.loc[game_id].tags)

Batman: Arkham Asylum Game of the Year Edition
['Action', 'Batman', 'Stealth', 'Adventure', 'Third Person', 'Superhero', 'Singleplayer', "Beat 'em up", 'Open World', 'Comic Book', 'Detective', 'Atmospheric', 'Story Rich', 'Fighting', 'Action-Adventure', 'Controller', 'Metroidvania', '3D Vision', 'Puzzle', 'Horror']


In [98]:
df_2.isna().sum()

hours       2637
id             0
app_name       0
tags           4
dtype: int64

In [99]:
dfx = df_2.dropna()

In [100]:
dfx.head()

,hours,id,app_name,tags
0,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,13.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,0.4,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,2.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


---

In [101]:
dfy = pd.merge(df_1,df_2, right_index=True, left_index=True)

In [102]:
dfy.sort_values('users', ascending=True)

,users,hours,id,app_name,tags
698212,0,6.5,386360,SMITE®,"[Free to Play, MOBA, Multiplayer, Action, Thir..."
497676,1,1522.2,230410,Warframe,"[Free to Play, Action, Co-op, Multiplayer, Thi..."
239234,1,3.1,327890,I am Bread,"[Simulation, Funny, Comedy, Physics, Adventure..."
96992,2,6.3,636040,Pixel Worlds,"[Free to Play, Adventure, Massively Multiplaye..."
208544,2,10.5,304050,Trove,"[Free to Play, Open World, Multiplayer, Advent..."
...,...,...,...,...,...
219945,537500,1.9,302830,BLOCKADE 3D,"[Early Access, Free to Play, Shooter, Multipla..."
238136,537501,27.9,99900,Spiral Knights,"[Free to Play, Massively Multiplayer, Adventur..."
263909,537501,6.1,337340,Finding Paradise,"[Story Rich, Great Soundtrack, Indie, Adventur..."
111145,537502,5.3,322330,Don't Starve Together,"[Survival, Multiplayer, Co-op, Open World, Adv..."


In [103]:
df_11 = dfy.truncate( after=10000)

In [104]:
df_11.tail()

,users,hours,id,app_name,tags
9996,188261,0.1,310370,The Way of Life Free Edition,"[Free to Play, Walking Simulator, First-Person..."
9997,224089,0.2,310370,The Way of Life Free Edition,"[Free to Play, Walking Simulator, First-Person..."
9998,537480,0.4,310370,The Way of Life Free Edition,"[Free to Play, Walking Simulator, First-Person..."
9999,193903,0.2,310370,The Way of Life Free Edition,"[Free to Play, Walking Simulator, First-Person..."
10000,153287,0.5,310370,The Way of Life Free Edition,"[Free to Play, Walking Simulator, First-Person..."


In [105]:
dfw = dfy.truncate( after=10000)

In [106]:
df_11['tags']=[" ".join(tags) for tags in df_11['tags'].values]

In [107]:
df_11.head()

,users,hours,id,app_name,tags
0,62225,0.1,725280,Psi Project,Action Adventure Indie Simulation
1,431962,0.1,725280,Psi Project,Action Adventure Indie Simulation
2,26750,13.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
3,422892,0.4,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
4,245491,2.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3


In [108]:
df_111 = df_11.drop(['hours', 'tags', 'app_name'], 1)

In [109]:
dr = pd.DataFrame(pd.qcut(df_11['hours'], 6, labels=['verybad', "bad", "medium", "good", 'verygood','Excelent']).dropna())

In [110]:
dr.head()

,hours
0,verybad
1,verybad
2,bad
3,verybad
4,verybad


In [111]:
dr2 = pd.DataFrame(dr.hours.astype("category").cat.codes)

In [112]:
dr2.head()

,0
0,0
1,0
2,1
3,0
4,0


In [113]:
dr2.columns = ['rating']

In [114]:
dr3 = pd.merge(dr2,df_111, right_index=True, left_index=True)

In [115]:
dr3.describe()

,rating,users
count,9994.000000,9994.000000
mean,2.498399,260580.262758
std,1.709003,152248.263116
min,0.000000,159.000000
25%,1.000000,130433.750000
50%,2.000000,257583.000000
75%,4.000000,383443.250000
max,5.000000,537480.000000


In [116]:
dr3 = dr3.dropna()

In [117]:
dr3.astype('str')

,rating,users,id
0,0,62225,725280
1,0,431962,725280
2,1,26750,328100
3,0,422892,328100
4,0,245491,328100
...,...,...,...
9996,0,188261,310370
9997,0,224089,310370
9998,0,537480,310370
9999,0,193903,310370


---

## Aplicamos la libreria Surprise

In [118]:
N_filas = 10000
data = Dataset.load_from_df(dr3[['users', 'id', 'rating']][:N_filas], reader)

In [119]:
from surprise.model_selection import train_test_split as tts

In [120]:
trainset, testset = tts(data, test_size=0.25)

In [121]:
from surprise import SVD

In [122]:
algo = SVD()

In [123]:
algo.fit(trainset)

In [124]:
predictions = algo.test(testset)

In [125]:
predictions[1]

Prediction(uid=447084, iid='271590', r_ui=4.0, est=3.471168173849336, details={'was_impossible': False})

In [126]:

from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 1.1864


1.1864149613827821

In [127]:
usuario = 26750
rating = 1 # le pedimos peliculas a las que haya puesto 4 o 5 estrellas
df_user = dr3[(dr3['users'] == usuario) & (dr3['rating'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user['app_name'] = df_11['app_name'].loc[df_user.rating].values
df_user

,rating,users,id,app_name
0,1,26750,328100,Psi Project


In [128]:
recomendaciones_usuario = df_11.iloc[:usuario].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(10001, 5)


,users,hours,id,app_name,tags
0,62225,0.1,725280,Psi Project,Action Adventure Indie Simulation
1,431962,0.1,725280,Psi Project,Action Adventure Indie Simulation
2,26750,13.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
3,422892,0.4,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
4,245491,2.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3


In [129]:
usuario_vistas = df_11[df_11['users'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

(1, 5)


,users,hours,id,app_name,tags
2,26750,13.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3


In [130]:
recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['id'].apply(lambda x: algo.predict(usuario, x).est)

In [131]:
print(f'Las recomendaciones para el usuario y sus scores son:\n')
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

Las recomendaciones para el usuario y sus scores son:

       users  hours      id            app_name  \
5000  265697   58.7  271590  Grand Theft Auto V   
4823  508282   71.9  271590  Grand Theft Auto V   
4849  116114   68.6  271590  Grand Theft Auto V   
4848  371006  103.1  271590  Grand Theft Auto V   
4847    2831   65.0  271590  Grand Theft Auto V   
4846   41539  159.0  271590  Grand Theft Auto V   
4845  232057   16.7  271590  Grand Theft Auto V   
4844  349489   66.7  271590  Grand Theft Auto V   
4843  277564    0.4  271590  Grand Theft Auto V   
4842  247051   52.7  271590  Grand Theft Auto V   

                                                   tags  Estimate_Score  
5000  Open World Action Multiplayer First-Person Thi...        3.596736  
4823  Open World Action Multiplayer First-Person Thi...        3.596736  
4849  Open World Action Multiplayer First-Person Thi...        3.596736  
4848  Open World Action Multiplayer First-Person Thi...        3.596736  
4847  Open Wo

---

## Multilabel Encoder y TFIDF

* Vamos hacer un multilabel encoder para los tags y vectorizarlos con TFIDF
* Luego vamos a haer una clasificacion logistica
* Finalmente obtendremos los géneros actuales y los mas importante los géneros que nos recomienda

In [132]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [133]:
dfw = pd.DataFrame(dfw)
dfw.head()

,users,hours,id,app_name,tags
0,62225,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,431962,0.1,725280,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,26750,13.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,422892,0.4,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,245491,2.2,328100,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


In [134]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(dfw['tags'])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [135]:
dfml = pd.DataFrame(mlb.fit_transform(dfw['tags']),columns=mlb.classes_)
dfml.head()

,1990's,2D,3D Platformer,3D Vision,4 Player Local,Abstract,Action,Action RPG,Action-Adventure,Adventure,...,Video Production,Violent,Visual Novel,Voice Control,Walking Simulator,War,Wargame,World War I,World War II,Zombies
0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [136]:
#list(mlb.classes_)

In [137]:
dfj = dfw

In [138]:
dfj['tags']=[" ".join(tags) for tags in dfw['tags'].values]
dfj.head()

,users,hours,id,app_name,tags
0,62225,0.1,725280,Psi Project,Action Adventure Indie Simulation
1,431962,0.1,725280,Psi Project,Action Adventure Indie Simulation
2,26750,13.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
3,422892,0.4,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
4,245491,2.2,328100,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3


In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [140]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [141]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(dfj['tags'], dfml, test_size=0.2, random_state=9)

In [142]:
# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [143]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [144]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [145]:
# fit model on train data
clf.fit(xtrain_tfidf, ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [146]:
y_pred = clf.predict(xval_tfidf)

In [147]:
y_pred[3]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Vemos un ejemplo de los generos recomendados

In [148]:
multilabel_binarizer.inverse_transform(y_pred)[2]

('Action',
 'Adventure',
 'Atmospheric',
 'Co-op',
 'Comedy',
 'Crime',
 'First-Person',
 'Funny',
 'Great Soundtrack',
 'Mature',
 'Moddable',
 'Multiplayer',
 'Open World',
 'RPG',
 'Racing',
 'Sandbox',
 'Shooter',
 'Singleplayer',
 'Third Person',
 'Third-Person Shooter')

In [149]:
f1_score(yval, y_pred, average="micro")

0.9979077594833076

In [150]:
y_pred_prob = clf.predict_proba(xval_tfidf)

In [151]:
t = 0.3 
y_pred_new = (y_pred_prob >= t).astype(int)

In [152]:
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    return text

In [153]:
def infer_tags(q):
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [154]:
for i in range(5): 
  k = xval.sample(1).index[0] 
  print("\n", "Juego:", dfx['app_name'][k], "\n", "\nGenero Predecido: ", infer_tags(xval[k])),"\n", "\n", print("\nGenero Actual: ",dfx['tags'][k], "\n")


 Juego: Back to Bed 
 
Genero Predecido:  [('Action', 'Casual', 'Indie', 'Isometric', 'Puzzle', 'Singleplayer', 'Surreal')]

Genero Actual:  ['Indie', 'Puzzle', 'Casual', 'Surreal', 'Action', 'Isometric', 'Singleplayer'] 


 Juego: Grand Theft Auto V 
 
Genero Predecido:  [('Action', 'Adventure', 'Atmospheric', 'Co-op', 'Comedy', 'Crime', 'First-Person', 'Funny', 'Great Soundtrack', 'Mature', 'Moddable', 'Multiplayer', 'Open World', 'RPG', 'Racing', 'Sandbox', 'Shooter', 'Singleplayer', 'Third Person', 'Third-Person Shooter')]

Genero Actual:  ['Open World', 'Action', 'Multiplayer', 'First-Person', 'Third Person', 'Crime', 'Adventure', 'Shooter', 'Third-Person Shooter', 'Singleplayer', 'Racing', 'Mature', 'Atmospheric', 'Sandbox', 'Co-op', 'Great Soundtrack', 'Funny', 'Comedy', 'Moddable', 'RPG'] 


 Juego: FINAL FANTASY® XIII-2 
 
Genero Predecido:  [('Action', 'Action RPG', 'Adventure', 'Anime', 'Atmospheric', 'Cinematic', 'Fantasy', 'Female Protagonist', 'Great Soundtrack', 'JRPG',

---

---

## Recomendación con TFIDF y Coseno de cercania

In [155]:
df_15 = df[['app_name','tags']]

In [156]:
df_15.head()

,app_name,tags
0,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


In [157]:
df_16 = df_15.truncate( after=30000)

In [158]:
df_16.rename(columns={'app_name':'title',
                          'tags':'tags'}, 
                 inplace=True)

In [159]:
df_16.head()

,title,tags
0,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


---

In [160]:
df_50 =  pd.merge(df_1,df_16, right_index=True, left_index=True)

In [161]:
df_50.head()

,users,title,tags
0,62225,Psi Project,"[Action, Adventure, Indie, Simulation]"
1,431962,Psi Project,"[Action, Adventure, Indie, Simulation]"
2,26750,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
3,422892,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"
4,245491,Gunspell - Steam Edition,"[Adventure, RPG, Indie, Puzzle, Match 3]"


---

In [162]:
df_16['tags']=[" ".join(tags) for tags in df_16['tags'].values]

In [163]:
df_16.head()

,title,tags
0,Psi Project,Action Adventure Indie Simulation
1,Psi Project,Action Adventure Indie Simulation
2,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
3,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3
4,Gunspell - Steam Edition,Adventure RPG Indie Puzzle Match 3


In [164]:
df_31 = df_16.groupby("title").sum()

In [165]:
df_32 = df_31.reset_index()

In [166]:
df_32.head()

,title,tags
0,A Room Beyond,Adventure Indie Point & Click
1,A Timely Intervention,Casual Adventure Indie RPG Strategy RPGMaker
2,A Valley Without Wind,Platformer Action Indie Adventure Procedural G...
3,About Elise,Free to Play Adventure Indie Casual Simulation...
4,Abscond,Indie Casual Action Singleplayer Atmospheric A...


---

## Generamos una matriz para los Géneros

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [168]:

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [169]:
tfidf_matrix = tf.fit_transform(df_32['tags'])

In [170]:
tfidf_matrix.shape

(321, 2700)

## Generamos un coseno de similitud 

In [171]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.14964352, 0.02009376, 0.13641173],
       [0.14964352, 1.        , 0.0163904 , 0.13528577],
       [0.02009376, 0.0163904 , 1.        , 0.01494113],
       [0.13641173, 0.13528577, 0.01494113, 1.        ]])

In [172]:
from sklearn.metrics.pairwise import cosine_similarity as cos

In [173]:
cos(tfidf_matrix[0].reshape(1,-1), tfidf_matrix[1].reshape(1,-1))

array([[0.14964352]])

In [174]:
titles = df_32['title']
indices = pd.Series(df_32.index, index=df_32['title'])

In [175]:
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores,key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

### Vamos a tomar como ejemplo un juego para ver si cada tipo de recomendación funciona de manera similar:

* El juego que vamos a elegir es:

                * Batman: Arkham Asylum Game of the Year Edition
                * id: 25
                * Genero: Stealth
                * User: 51526

---

## Recomendación por Juego

* Se puede seleccionar un juego y buscar de la recomendación 

* Por ejemplo: 

                * Batman: Arkham Asylum Game of the Year Edition
                * iLawBreakers
                * Who's Your Daddy

In [176]:
game_play = 'Batman: Arkham Asylum Game of the Year Edition'
rp = pd.DataFrame(df_32[df_32['title'].str.match(game_play)].sample().iloc[0:1].title).iloc[0]['title']
print(f'La recomendación para {game_play} es: \n')
genre_recommendations(rp).head(20)

La recomendación para Batman: Arkham Asylum Game of the Year Edition es: 



238    Scribblenauts Unmasked: A DC Comics Adventure
151                                     LUMBERMANCER
44                     Brothers - A Tale of Two Sons
97                                     Freedom Force
203                                Prince of Persia®
22                           Back to Dinosaur Island
227                        Rumble Fighter: Unleashed
129                                 House of Caravan
132                                       I Am Alive
35                                        Black Mist
71                          Deus Ex: Mankind Divided
121                                      Heart&Slash
174                                      Monsterland
166                                         MicroSpy
17                                         Ascendant
42                                        Brawlhalla
275                                     The Novelist
278                     The Way of Life Free Edition
183                        Need for Speed Unde

---

## Recomendación por ID

* Se puede seleccionar el ID del juego y ver su recomendación

* Por ejemplo: 

                * id: 10 /  Alea Jacta Est 
                * id: 243 / Simulator hipstera 2k17
                * id: 302 / WayOut 2: Hex

In [177]:
game_id = 34
result_game = df_32.loc[game_id].title
print(f'El juego del id número {game_id} es: {result_game}\n')
genre_recommendations(result_game).head(20)

El juego del id número 34 es: Birthdays the Beginning / バースデイズ・ザ・ビギニング



128                                  Hospital Tycoon
245                                        Snakebird
257                         Stratego - Single Player
58                                Creature Factory 2
54                  Commands & Colors: The Great War
200                                    Poker Night 2
282                         Tomb Raider: Anniversary
22                           Back to Dinosaur Island
104                                  Ghost of a Tale
18                                           Asteria
19                                     Astral Heroes
250                                       Spellstone
186                                           Nyheim
295                  VEmpire - The Kings of Darkness
114                                   Gremlins, Inc.
48                                   Castle Explorer
67                                   Dead Man's Draw
56                                   Costume Quest 2
124    Higurashi When They Cry Hou - Ch.1 Onik

---

## Recomendación por género

* Se pueden seleccionar distintos tipos de generos para generar la recomendación. 

* Por ejemplo: 

                * 'Racing'
                * 'Adventure'
                * 'Indie'
                * 'Singleplayer'
                * 'RPG' 

In [178]:
game_genre = 'Action'
rg = pd.DataFrame(df_32[df_32['tags'].str.match(game_genre)].sample().iloc[0:1].title).iloc[0]['title']
print(f'El juego del genero {game_genre} es: {rg}\n')
genre_recommendations(rg).head(20)

El juego del genero Action es: Wooden Sen'SeY



157             Lost Socks: Naughty Brothers
69                                Deadly Sky
5                                Action Henk
171                                     Mint
176                My Coloring Book: Animals
261                              Superstatic
11                            All Evil Night
237                                Sacraboar
296                               VS Round 1
235                             STEEL RIVALS
35                                Black Mist
101                         Galactic Fighter
276                          The Return Home
88     Fairies vs. Darklings: Arcane Edition
43                         Breakout Invaders
166                                 MicroSpy
229                                   Runbow
317                                   gShift
263                                  TRISTOY
228                           Run Rabbit Run
Name: title, dtype: object

---

## Recomendación por User

* Se puede seleccionar un user y buscar de la recomendación 

* Por ejemplo: 

                * El user: 51526 jugo Batman: Arkham Asylum Game of the Year Edition 
            

In [179]:
game_users = '62225'
df_51 = pd.DataFrame(df_50).applymap(str)
ru = pd.DataFrame(df_51[df_51['users'].str.match(game_users)].title).iloc[0]['title']
print(f'El juego del genero {game_users} es: {ru}\n')
genre_recommendations(ru).head(20)

El juego del genero 62225 es: Psi Project



35                                Black Mist
166                                 MicroSpy
317                                   gShift
151                             LUMBERMANCER
105          Ghoulboy - Dark Sword of Goblin
243                  Simulator hipstera 2k17
147                             Jaques Roque
268         The Bits That Saved The Universe
236                   STLD Redux: Episode 02
76     Draft Day Sports: Pro Basketball 2017
228                           Run Rabbit Run
0                              A Room Beyond
6                             Age of Farming
189             Off-Road Paradise: Trial 4x4
3                                About Elise
1                      A Timely Intervention
72                               Diamond Dan
90                    Farm Frenzy Collection
73                              Diaper Dash®
140                           Indie Game Sim
Name: title, dtype: object

---

## Conclusión

* Realizamos un sistema de recomendación Utilizando la libreria Surprise, lo cual nos da la posibilidad de:

        * Obtener resultados apra el juego seleccionado
        * Nos otorga el valor predicho de cada una de las recomendaciones.

* Como segunda opción realizamos un sistema de recomendación utilizando la TFIDF y Coseno de similitud, y nos da la posibilidad de:

        
        * Recomendaciones por ID de juego
        * Recomendaciones por género de juego

* Como tercera opción realizamos un sistema de recomendación utilizando la Multilabel encoder y TFIDF para darnos la posibilidad de: 

        * Recomendaciones por género
        * Recomendaciones por título de juego
        

* Sumamos los accuracy de cada sistema:
    * Surprise:

    

In [180]:
accuracy.rmse(predictions)

RMSE: 1.1864


1.1864149613827821

* Multilabel Encoder y TFIDF:

In [181]:
f1_score(yval, y_pred, average="micro")

0.9979077594833076

* Coseno de Similitud y TFIDF:

In [182]:
cos(tfidf_matrix[0].reshape(1,-1), tfidf_matrix[1].reshape(1,-1))

array([[0.14964352]])

---

# thanks!